# How to use honomara_web_env

honomara_web_envは、ホノルルマラソンを走る会のホームページ作成のための統合環境です。

## 1. jupyter上でのbashの操作になれる

このノートはjupyter notebookというソフトウェア上で動作しています。  
jupyter notebookというのは、人間に読みやすい形でにプログラムを実行、記録できるサービスです。  
通常はpythonを実行するために使われることが多いですが、今回は環境構築のために、bash（ターミナル環境）を使えるようにしています。  

In [ ]:
#まずは実行してみましょう
#枠の左側の再生ボタンをクリックするか、
#枠を選択した状態で Shift + Enter または Ctrl + Enterで
#枠内のコマンド（コード）を実行できます。 
echo "現在のディレクトリ"
pwd

正しく実行できたらセル（枠）の左の\[\]内に数字が表示され、

現在のディレクトリ  
/home/notebooks  

と表示されるはずです。

次に別のコマンドを実行してみましょう

In [ ]:
# Shift + Enter 等で実行してください
ls /vagrant

/vagrant中のディレクトリのファイルやディレクトリ一覧が表示されます。  
気づいた人もいるかもしれませんが、  
このディレクトリ（仮想マシン中の/vagrantディレクトリ）はホスト中のダウンロードしてきたリポジトリと同期するように設定してあります。  

複数のセル（それぞれの枠のこと）を用意したので、好きなようにコマンドを実行してみてください  
セルを追加はメニューの`insert`から可能です。  
ショートカットでもできるので、`Help`を参照してみてください。

In [ ]:
# Shift + Enter 等で実行する。
date

## 2. 各環境をセットアップする

WEB開発に関係する可能性のある様々な環境を構築する手段（スクリプト）を用意しています。  
ラインナップは以下のとおりです。
<table>
    <thead><td>項目</td><td>説明</td><td>インストール方法</td> </thead>
    <tr><td>pytho3</td><td>python3.5</td><td>(インストール済み)</td></tr>
    <tr><td>pip3</td><td>python3パッケージ管理用</td><td>(インストール済み)</td></tr>
    <tr><td>Jupyter</td><td>jupyter</td><td>(インストール済み)</td></tr>
    <tr><td>PostgreSQL</td><td>データベース</td><td>/vagrant/scripts/setup_postgres.sh</td></tr>
    <tr><td>MySQL</td><td>データベース</td><td>/vagrant/scripts/setup_mysql.sh</td></tr>
    <tr><td>Apache2&PHP</td><td>WEBサーバーとPHP7.3</td><td>/vagrant/scripts/setup_web_server.sh</td></tr>
    <tr><td>MeCab</td><td>形態素解析機</td><td>/vagrant/scripts/setup_mecab.sh</td></tr>    
    <tr><td>slim</td><td>PHP の軽量WEBフレームワーク</td><td>/vagrant/scripts/setup_slim.sh</td></tr>
    <tr><td>Flask</td><td>Python の軽量WEBフレームワーク</td><td>/vagrant/scripts/setup_flask.sh</td></tr>
    <tr><td>既存データ</td><td>PostgreSQL上にホノマラのデータをインポート</td><td>/vagrant/scripts/import_honomara_data.sh</td></tr>
</table>

### 2.1 PostgreSQLのインストール〜〜過去のデータを見てみよう〜〜
#### 2.1.1 PostgreSQLのインストール

In [ ]:
# Shift + Enter 等で実行してください
/vagrant/scripts/setup_postgres.sh

最後の行に
`/vagrant/notebooks`
と表示されたら成功です。

#### 2.1.2 過去のデータのインポート

既存のデータをデータベースに導入するためににスクリプトを用意したので実行してください

In [ ]:
# Shift + Enter 等で実行してください。
# 実行完了まで数分かかります。
/vagrant/scripts/import_honomara_data.sh

INSERT 0 1 
と大量に表示されるはずです
この処理には5分くらいかかると思います。
（セルの左側の\[\]の中が＊ではなく数字になったら終了です。）
それでは実際にデータを見てみましょう

In [ ]:
# Shift + Enter 等で実行してください
export PGPASSWORD=honomara
psql -U honomara -d honomara -c 'SELECT * FROM person LIMIT 10;'

それっぽいデータが見えたら成功です。

### 2.2 WEBサーバーのインストール 

今回は、古くから使われておりサークルのWEBサーバーでも使われているApacheというソフトとPHPをインストールします。

※
「サーバー」という単語ですが、実は複数意味があります。  
通常は「データセンターにおいてあるマシン」等を想像するかもしれません。  
ただここでは、「ユーザー（クライアント）からリクエストを受け取って応答するソフトウェア」という意味で使っています。  

In [ ]:
# Shift + Enter 等で実行してください。
# 実行完了まで数分かかります。
/vagrant/scripts/setup_web_server.sh

インストールできたら（セルの左の\[\]中に数字が表示されたら）ブラウザから

[127.0.0.1:8080](http://127.0.0.1:8080)
（または[localhost:8080](http://localhost:8080)）

を開いてください。恐らく  

Apache2 Ubuntu Default Page

と書いたページが表示されるはずです。

これは  
仮想マシン中の/var/www/html/index.html  
というファイルを表示したものです。

自分で作成したページを表示したい場合は
クローンしてきたリポジトリのディレクトリ(honomara_web_env)にHTML,CSS,JSのファイルを置いてください。  
このディレクトリは仮想マシン中で/vagrantディレクトリと同期しているので  
そこから/var/www/htmlディレクトリにデータを移動しましょう。  
以下例です。

実行してみましょう。

In [ ]:
cat > /vagrant/test.html <<EOF
<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <title>テストタイトル</title>
</head>
<body>
    <h1>これはテストです！！</h1>
</body>
</html>
EOF
cp  /vagrant/test.html /var/www/html/

実行したら

[127.0.0.1:8080/test.html](http://127.0.0.1:8080/test.html)

を開いてみましょう

これはテストです！！

と大きく表示されていたら成功です。

### 2.3 MySQLの設定

In [ ]:
/vagrant/scripts/setup_mysql.sh

### 2.4MeCabの設定

In [ ]:
/vagrant/scripts/setup_mecab.sh

### 2.5cgiの設定 （[参考リンク](https://www.server-world.info/query?os=Ubuntu_16.04&p=httpd&f=5)）

In [ ]:
sudo a2enmod cgi
sudo systemctl restart apache2
[ -d /usr/lib/cgi-bin ] || sudo rmdir /usr/lib/cgi-bin
[ -d /vagrant/app ] || mkdir -p /vagrant/app
[ -e /usr/lib/cgi-bin ] || sudo ln -s /vagrant/app /usr/lib/cgi-bin
sudo chmod 777 /usr/lib/cgi-bin
cat > /usr/lib/cgi-bin/test_script <<'EOF'
#!/usr/bin/env python3
print("Content-type: text/html\n\n")
print("Hello CGI\n")
EOF
chmod 705 /usr/lib/cgi-bin/test_script

### 2.6Flaskのための各種設定

In [ ]:
sudo pip3 install flask
sudo sed -i -E 's/(AllowOverride\s+)None/\1All/g' /etc/apache2/apache2.conf
sudo sed -i -E 's/(AllowOverride\s+)None/\1All/g' /etc/apache2/conf-enabled/serve-cgi-bin.conf
sudo a2enmod rewrite
sudo systemctl restart apache2

cat > /var/www/html/.htaccess <<EOF
RewriteEngine On
RewriteRule ^cgi-bin\$ /cgi-bin/index.cgi [L,QSA]
EOF
chmod 604 /var/www/html/.htaccess

## 3.データの移行

以下、別のノートにデータ変換スクリプトをおいてあるので、実行してみてください。

1. [members](members.ipynb)
    * mecabの設定が必要です、あらかじめ/vagrant/scripts/setup_mecab.shを実行してください
    * カナの更新には、CSVの読み込みが必要です、スプレッドシートからCSV形式でダウンロードしてください。
2. [trainings](trainings.ipynb)
3. [afters](afters.ipynb)
4. [participants](participants.ipynb)